# Lab2 - Data Collection and Pre-processing-Jahnavi Pakanati-9013742

# Importing the required libraries 

In [38]:
import pandas as pd
from dataclasses import dataclass


# Loading the  raw CSV, displaying the  first 3 rows

In [39]:
df = pd.read_csv("data/clothing_transactions.csv")
print("Columns:", df.columns.tolist())
print(df.head(20))


Columns: ['date', 'customer_id', 'product', 'price', 'quantity', 'coupon_code', 'shipping_city', 'postal_code']
          date  customer_id   product   price  quantity coupon_code  \
0   2025-05-11         1000     Scarf  116.39         1     JEANS10   
1   2025-05-03         1001   T-Shirt   90.00         4     WINTER5   
2   2025-05-02         1002     Dress   80.00         2  SPRINGSALE   
3   2025-05-11         1003    Shorts   37.67         5         NaN   
4   2025-05-06         1004  Sneakers   52.64         4    SUMMER15   
5   2025-05-08         1005    Jacket  100.86         2    SUMMER15   
6   2025-05-11         1006     Dress   40.76         4    SUMMER15   
7   2025-05-06         1007     Scarf   68.01         3     CLOTH20   
8   2025-05-07         1008    Hoodie  108.53         1         NaN   
9   2025-05-08         1009     Dress   66.90         4  SPRINGSALE   
10  2025-05-04         1010    Jacket   98.25         2     WINTER5   
11  2025-05-11         1011    Short

#### Justify dict vs namedtuple vs class (1–2 sentences)
#### Dictionary--A flexible way to store data using key-value pairs.
#### NamedTuple--Like a tuple, but you can access values using names.
#### Class--A way to group data and functions together.

##### Class is the best option for me because it includes logics like cleaning, transforming and computing totals. So this will allow me to show better modularity and resuability for each transaction object.


##### Implementing the transaction class and using to populate 

In [40]:
@dataclass
class Transaction:
    date: str
    customer_id: str
    product: str
    price: float
    quantity: int
    coupon_code: str
    shipping_city: str
    postal_code: str

    


##### creating a method as load_transaction  and returning the list of transactions 

In [ ]:
def load_transactions(path: str) -> list[Transaction]:
    df = pd.read_csv(path)
    return [
        Transaction(
            date=row["date"],
            customer_id=row["customer_id"],
            product=row["product"],
            price=row["price"],
            quantity=row["quantity"],
            coupon_code=row["coupon_code"],
            shipping_city=row["shipping_city"],
            postal_code=row["postal_code"],
        ) for _, row in df.iterrows()
    ]


SyntaxError: invalid syntax. Perhaps you forgot a comma? (2667166240.py, line 11)

##### Creatiing method called profile_transactions_summary for gettting min/max/mean of price and unique city. It is taking the 

In [ ]:
def profile_transactions_summary(transactions: list) -> None:
    prices = [float(t.price) for t in transactions if isinstance(t.price, (int, float))]
    cities = {t.shipping_city for t in transactions}
    
    print("Step 5: Quick Profiling")
    print(f"Price Stats - Min: {min(prices)}, Mean: {sum(prices)/len(prices):.2f}, Max: {max(prices)}")
    print(f"Unique Shipping Cities: {len(cities)}\n")



In [ ]:
transactions = load_transactions("data/clothing_transactions.csv")
profile_transactions_summary(transactions)

TypeError: Transaction.__init__() missing 1 required positional argument: 'postal_code'